In [40]:
!pip install dotenv

In [41]:
!pip install pandas
import pandas as pd

In [42]:
import requests
from datetime import datetime, timedelta
from pytz import timezone
import time
import dotenv
import os 

### Creating Dataframe with Airports info extracted using API

In [43]:
dotenv.load_dotenv()

True

In [44]:
schema = "ganz"
host = "127.0.0.1"
user = os.environ['user_name']
password = os.environ['MYSQL_password']
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [45]:
def airports(lat, lon):               
                                        #Call this function with list of lat and lon even if it's just 1 value.

  url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

  headers = {
        "x-rapidapi-key": os.environ['Airports_Flights_API'],
        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }
  
  # Initializing airport_df to an empty DataFrame to ensure it's always defined
  city_airport_df = pd.DataFrame() 
  airports_list= []
  for lat, lon in zip(lat, lon):

      #querystring - lat and lon values were being passed incorrectly before
      querystring = {"lat":lat,"lon":lon,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}
      
      # Add a fixed 3-second delay before each request
      time.sleep(3)  # Sleep for 3 seconds before sending the request

      response = requests.get(url, headers=headers, params=querystring)
      
      if response.status_code == 200:
        airport_json = response.json() # Getting the json data from the response
        airport_df= pd.json_normalize(airport_json.get('items',[]))
        airport_df.rename(columns= {'municipalityName':'City', 'name':'Airport_Name', 'shortName':'Short_Name', 'icao':'ICAO','countryCode':'Country_Code'
                                   , 'location.lat': 'Location_lat', 'location.lon':'Location_lon'}, inplace= True)
  
        #Append
        airports_list.append(airport_df)
  #Conactinate the data of all cities
  city_airport_df = pd.concat(airports_list, ignore_index=True)


  return (city_airport_df)

In [46]:
cities= pd.read_sql('SELECT City_id, City, Latitude, Longitude FROM cities', con= connection_string)

In [47]:
lat= cities['Latitude'].to_list()
lon= cities['Longitude'].to_list()
lat,lon

([52.52, 53.55, 48.1375], [13.405, 10.0, 11.575])

In [48]:
airports_df= airports(lat,lon)

In [49]:
airports_df.drop(columns= ['iata','timeZone'], inplace= True)

In [50]:
airports_df

,ICAO,Airport_Name,Short_Name,City,Country_Code,Location_lat,Location_lon
0,EDDT,Berlin -Tegel,-Tegel,Berlin,DE,52.55970,13.287699
1,EDDB,Berlin Brandenburg,Brandenburg,Berlin,DE,52.35139,13.493889
2,EDDH,Hamburg,Hamburg,Hamburg,DE,53.63040,9.988229
3,EDDM,Munich,Munich,Munich,DE,48.35380,11.786100


In [51]:
airports_new_df= airports_df.merge(cities, how= 'inner', on= 'City')
airports_new_df.drop(columns= ['Latitude','Longitude'], inplace= True)

### Pushing info to SQL database

In [52]:
airports_new_df.to_sql('airports',
                   if_exists= 'append',
                   con= connection_string,
                      index= False                   
)

4

### Creating Database with flights info extracted using API:

In [53]:
def tomorrows_flight_arrivals(icao_list):
    api_key = os.environ['Airports_Flights_API']
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = today + timedelta(days=1)

    list_for_arrivals_df = []

    for icao in icao_list:
        print(f"Processing ICAO: {icao}")

        times = [["00:00", "11:59"], ["12:00", "23:59"]]

        for time_period in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time_period[0]}/{tomorrow}T{time_period[1]}"

            querystring = {"direction": "Arrival", "withCancelled": "false"}

            headers = {
                "X-RapidAPI-Key": api_key,
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)

            if response.status_code == 429:
                print(f"Rate limit exceeded for ICAO {icao}. Waiting before retrying...")
                retry_after = response.headers.get('Retry-After')
                if retry_after:
                    time.sleep(int(retry_after))
                else:
                    time.sleep(60)  # Default to 60 seconds if 'Retry-After' is not provided
                response = requests.get(url, headers=headers, params=querystring)
            
            if response.status_code != 200:
                print(f"Error with response for ICAO {icao}: {response.status_code}")
                continue

            try:
                flights_resp = response.json()
            except ValueError as e:
                print(f"Failed to decode JSON for ICAO {icao}: {e}")
                continue

            if 'arrivals' in flights_resp and flights_resp['arrivals']:
                arrivals_df = pd.json_normalize(flights_resp["arrivals"])[
                    ["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
                arrivals_df = arrivals_df.rename(columns={
                    "number": "Flight_number",
                    "airline.name": "Airline",
                    "movement.scheduledTime.local": "Local_arrival_time",
                    "movement.terminal": "Arrival_terminal",
                    "movement.airport.name": "Departure_city",
                    "movement.airport.icao": "Departure_airport_ICAO"
                })
                arrivals_df["Arrival_airport_ICAO"] = icao
                arrivals_df["Data_retrieved_on"] = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

                arrivals_df["Local_arrival_time"] = arrivals_df["Local_arrival_time"].str.split("+").str[0]

                list_for_arrivals_df.append(arrivals_df)
            else:
                print(f"No arrivals found for ICAO {icao}")

    if list_for_arrivals_df:
        return pd.concat(list_for_arrivals_df, ignore_index=True)
    else:
        print("No arrivals data found for any ICAOs.")
        return pd.DataFrame()


In [54]:
icaos = pd.read_sql('select ICAO from airports', con= connection_string)


In [55]:
icao_list= icaos['ICAO'].to_list()
icao_list

['EDDB', 'EDDT', 'EDDH', 'EDDM']

In [56]:
flights_df= tomorrows_flight_arrivals(icao_list)

Processing ICAO: EDDB
Processing ICAO: EDDT
Error with response for ICAO EDDT: 204
Error with response for ICAO EDDT: 204
Processing ICAO: EDDH
Processing ICAO: EDDM


In [57]:
flights_df

,Flight_number,Airline,Local_arrival_time,Arrival_terminal,Departure_city,Departure_airport_ICAO,Arrival_airport_ICAO,Data_retrieved_on
0,XQ 966,Sun Express,2025-04-06 06:30,1,İzmir,LTBJ,EDDB,2025-04-05 19:11:03
1,IV 7280,Caspian Airlines,2025-04-06 06:40,1,Prishtina,BKPR,EDDB,2025-04-05 19:11:03
2,EW 8995,Eurowings,2025-04-06 06:40,1,Yerevan,UDYZ,EDDB,2025-04-05 19:11:03
3,HU 489,Hainan,2025-04-06 06:45,1,Beijing,ZBAA,EDDB,2025-04-05 19:11:03
4,XQ 658,Sun Express,2025-04-06 06:45,1,Antalya,LTAI,EDDB,2025-04-05 19:11:03
...,...,...,...,...,...,...,...,...
1083,LH 124,Lufthansa,2025-04-06 23:10,2,Frankfurt-am-Main,EDDF,EDDM,2025-04-05 19:11:08
1084,DE 1413,Condor,2025-04-06 23:15,1D,Fuerteventura Island,GCFV,EDDM,2025-04-05 19:11:08
1085,TP 556,TAP Air Portugal,2025-04-06 23:20,2,Lisbon,LPPT,EDDM,2025-04-05 19:11:08
1086,LH 2481,Lufthansa,2025-04-06 23:20,2,London,EGLL,EDDM,2025-04-05 19:11:08


### Pushing info to MYSQL

In [58]:
flights_df.to_sql('flights',
                  if_exists= 'append',
                  index= False,
                  con= connection_string
)

1088